# Méthode de Newton

In [1]:
using Plots
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots C:\Users\slash\.julia\packages\Plots\6RLiv\src\backends.jl:373


Plots.PlotlyBackend()

## Newton-Raphson en optimisation

Itération de Newton-Raphson:
$$
x_{k+1} = x_k-\frac{f'(x_k)}{f''(x_k)}
$$

In [3]:
function Newton(f::Function, df::Function, d2f:: Function,
        xstart::Float64, verbose::Bool = false, store=false,
        δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    x = xstart
    if (store)
        iter = [ f(x) x ]
    end
    if (verbose)
        fx = f(x)
        println("$k. x = $x, f(x) = $fx")
    end
    dfx = df(x)
    while (abs(dfx) > δ && k < nmax)
        k += 1
        dfx = df(x)
        x = x-dfx/d2f(x)
        if (store)
            iter = [iter ; f(x) x]
        end
        if (verbose)
            fx = f(x)
            println("$k. x = $x, f(x) = $fx")
        end
    end
    
    if (store)
        return iter
    end
    
    return x
end

Newton (generic function with 5 methods)

In [4]:
func(x) = -10x^2 + 4sin(x) + x^4
dfunc(x) = -20x + 4cos(x) + 4x^3
d2func(x) = -20 - 4sin(x) + 12x^2

d2func (generic function with 1 method)

In [5]:
plot(func, -4.0, 4.0)

<!DOCTYPE html>
 
 
 Plots.jl

Si nous commençons assez proche de l'optimum global, nous pouvons le trouver.

In [6]:
x0 = -3.0
iter = Newton(func, dfunc, d2func, x0, true, true)

1. x = -3.0, f(x) = -9.564480032239473
2. x = -2.413309150943051, f(x) = -26.983281625936534
3. x = -2.2051283480850294, f(x) = -28.202989762523757
4. x = -2.1772606979392806, f(x) = -28.219314490011172
5. x = -2.1767739238815578, f(x) = -28.21931925035788
6. x = -2.1767737764195543, f(x) = -28.21931925035831
7. x = -2.176773776419541, f(x) = -28.219319250358318


7×2 Array{Float64,2}:
  -9.56448  -3.0
 -26.9833   -2.41331
 -28.203    -2.20513
 -28.2193   -2.17726
 -28.2193   -2.17677
 -28.2193   -2.17677
 -28.2193   -2.17677

In [7]:
iter

7×2 Array{Float64,2}:
  -9.56448  -3.0
 -26.9833   -2.41331
 -28.203    -2.20513
 -28.2193   -2.17726
 -28.2193   -2.17677
 -28.2193   -2.17677
 -28.2193   -2.17677

In [12]:
plot(func, -3.2, -2.0, label="Function")

<!DOCTYPE html>
 
 
 Plots.jl

In [13]:
x = -3.0
m(y) = func(x)+dfunc(x)*(y-x)+0.5*d2func(x)*(y-x)^2
m(-3.0)

-9.564480032239473

In [9]:
plot!(m, -3.2, -2.0, label="Model")
plot!(iter[1:2,2], iter[1:2,1], label="Newton step")
vline!([iter[1,2] iter[2,2]], label = "")

<!DOCTYPE html>
 
 
 Plots.jl

In [10]:
plot(func, -3.2, -2.0, label="Function")
plot!(iter[:,2], iter[:,1], label="Newton steps")

<!DOCTYPE html>
 
 
 Plots.jl

In [14]:
x0 = -4.0
Newton(func, dfunc, d2func, x0, true)

1. x = -4.0, f(x) = 99.02720998123172
2. x = -2.942938833739946, f(x) = -12.3872911024757
3. x = -2.3879767853105314, f(x) = -27.24370765699775
4. x = -2.199836344922852, f(x) = -28.20853988049366
5. x = -2.177097516735148, f(x) = -28.219317146168752
6. x = -2.1767738416160567, f(x) = -28.21931925035823
7. x = -2.176773776419543, f(x) = -28.219319250358318
8. x = -2.1767737764195405, f(x) = -28.219319250358314


-2.1767737764195405

In [15]:
x0 = -2.2
Newton(func, dfunc, d2func, x0, true)

1. x = -2.2, f(x) = -28.208385615278356
2. x = -2.177102076816536, f(x) = -28.219317086469566
3. x = -2.176773843465362, f(x) = -28.219319250358225
4. x = -2.1767737764195436, f(x) = -28.219319250358314
5. x = -2.176773776419541, f(x) = -28.219319250358318


-2.176773776419541

Cependant, si la méthode converge, nous savons seulement qu'elle converge vers un point où la dérivée s'annule. Il pourrait s'agit d'un maximum local!

In [16]:
x0 = 1.0
Newton(func, dfunc, d2func, x0, true)

1. x = 1.0, f(x) = -5.634116060768414
2. x = -0.21757276869246645, f(x) = -1.3345792339543026
3. x = 0.22489446800386675, f(x) = 0.38879682400433235
4. x = 0.19762712402741728, f(x) = 0.3963333960713819


0.19765040255668145

5. x = 0.19765040254573119, f(x) = 0.3963334015760846
6. x = 0.19765040255668145, f(x) = 0.39633340157608465


In [17]:
α = 0.1
x = x0-α*dfunc(x0)/d2func(x0)
α = 0.001
x = x0-α*dfunc(x0)/d2func(x0)

0.9987824272313075

In [19]:
func(x)

-5.617274757402128

In [20]:
dfunc(x0)/d2func(x0)

1.2175727686924664

In [21]:
d2func(x0)

-11.365883939231587

In [25]:
x = 1.0
m(y) = func(x)+dfunc(x)*(y-x)+0.5*d2func(x)*(y-x)^2
plot(m, -1.0, 1.5, label="Model")

<!DOCTYPE html>
 
 
 Plots.jl

Nous pouvons aussi converger vers un minimum local, mais pas global.

In [26]:
x0 = 2.0
Newton(func, dfunc, d2func, x0, true)

1. x = 2.0, f(x) = -20.362810292697276
2. x = 2.3966942741858284, f(x) = -21.73466210734035
3. x = 2.3059655951195412, f(x) = -21.932339213110737
4. x = 2.299954353227294, f(x) = -21.933079241292685
5. x = 2.299928487772611, f(x) = -21.933079254838706
6. x = 2.2999284872946024, f(x) = -21.93307925483871


2.2999284872946024

In [28]:
x = 2.0
m(y) = func(x)+dfunc(x)*(y-x)+0.5*d2func(x)*(y-x)^2
plot(m, 0.0, 3.5, label="Model")

<!DOCTYPE html>
 
 
 Plots.jl

## Méthode de la sécante

In [29]:
function Secant(f::Function, df::Function, x0::Float64, x1::Float64,
                verbose::Bool = false, δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    x = x0
    y = x1
    if (x0 == x1)
        println("x0 must different from x1")
        return
    end
    if (verbose)
        println("0. x0 = $x0, f($x0) = $(f(x0))")
        println("1. x1 = $x1, f($x1) = $(f(x1))")
    end
    dfx = df(x)
    dfy = df(y)
    while (abs(dfy) > δ && k < nmax)
        k += 1
        t = y
        y = y-(x-y)/(dfx-dfy)*dfy
        x = t
        if (verbose)
            println("$k. x = $y, f(x) = $(f(y))")
        end
        dfx = dfy
        dfy = df(y)        
    end
    
    return y
end

Secant (generic function with 4 methods)

In [30]:
Secant(func, dfunc, 1.0, 2.0, true)

0. x0 = 1.0, f(1.0) = -5.634116060768414
1. x1 = 2.0, f(2.0) = -20.362810292697276
2. x = 4.315312971079605, f(x) = 156.8680861524251
3. x = 2.091990211601243, f(x) = -21.142250406961065
4. x = 2.158559714071919, f(x) = -21.55524618694635
5. x = 2.3260276821626857, f(x) = -21.919116004114606
6. x = 2.297113186106924, f(x) = -21.9329189976418
7. x = 2.2998766649378366, f(x) = -21.93307920046687
8. x = 2.2999285917018053, f(x) = -21.93307925483849
9. x = 2.2999284872907366, f(x) = -21.93307925483871


2.2999284872907366

## Recherche de racine avec la méthode de Newton

Itération:
$$
x_{k+1} = x_k-\frac{f(x)}{f'(x)}
$$

Considérons la fonction

In [31]:
f(x) = x-2sin(x)

f (generic function with 1 method)

In [32]:
df(x) = 1-2cos(x)

df (generic function with 1 method)

In [33]:
plot(f, -5.0, 5.0)

<!DOCTYPE html>
 
 
 Plots.jl

Nous cherchons un zéro de $f(x)$.

In [34]:
function NewtonRoot(f::Function, df::Function, xstart::Float64,
        verbose::Bool = false, δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    x = xstart
    fx = f(x)
    if (verbose)
        println("$k. x = $x, f(x) = $fx")
    end
    while (abs(fx) > δ && k < nmax)
        k += 1
        dfx = df(x)
        x -= fx/df(x)
        fx = f(x)
        if (verbose)
            println("$k. x = $x, f(x) = $fx")
        end
    end
    
    return x
end

NewtonRoot (generic function with 4 methods)

In [35]:
x0 = 1.1
NewtonRoot(f, df, x0, true)

1. x = 1.1, f(x) = -0.6824147201228707
2. x = 8.45299226150258, f(x) = 6.801204556312422
3. x = 5.256413636974667, f(x) = 6.96767867623376
4. x = 203.38418371898206, f(x) = 201.92279459710403
5. x = 118.01933072340529, f(x) = 119.97555657845707
6. x = -87.47086925902863, f(x) = -88.41868794280741
7. x = -203.63588782873876, f(x) = -202.56060951992214
8. x = -128.23223445457745, f(x) = -127.14781568637801
9. x = -80.79763601602076, f(x) = -82.34391070430607
10. x = -387.5048303996056, f(x) = -389.27709720983694
11. x = -185.47821725451567, f(x) = -185.7260797295505
12. x = -123.24970298358457, f(x) = -124.5798458868416
13. x = -73.28905518426131, f(x) = -75.00614412022145
14. x = -36.257729314803385, f(x) = -38.24100474494402
15. x = 15.287357802605989, f(x) = 14.470731358617773
16. x = 10.166214566380745, f(x) = 11.516910773438564
17. x = 5.512913942179146, f(x) = 6.9055739998247505
18. x = 21.37162909095707, f(x) = 20.210341066972575
19. x = 13.682162586651351, f(x) = 11.8856443575909

8.746965012105395e-14

Cela fonctionne, mais nous étions proches d'un désastre! Observons que

In [36]:
df(1.1)

0.09280775714884537

La courbe est presque plate, et nous sommes entraînés vers un point très éloigné. Par chance, nous revenons sur nos pas, mais en utilisant beaucoup d'itérations.

Considérons à présent le point de départ

In [37]:
x0 = π/3

1.0471975511965976

La dérivée à ce point est

In [38]:
df(x0)

-2.220446049250313e-16

La méthode de Newton donne

In [39]:
NewtonRoot(f, df, x0, true)

1. x = 1.0471975511965976, f(x) = -0.6848532563722796
2. x = -3.084304870201668e15, f(x) = -3.084304870201667e15
3. x = -8.117394305716909e15, f(x) = -8.11739430571691e15
4. x = -2.1048257039058684e16, f(x) = -2.1048257039058684e16
5. x = -4.805339681889308e15, f(x) = -4.805339681889306e15
6. x = -3.5972267403992e13, f(x) = -3.5972267403993945e13
7. x = 3.2802041789131953e13, f(x) = 3.280204178912997e13
8. x = 6.219979879306691e12, f(x) = 6.219979879308627e12
9. x = -6.303602849041078e12, f(x) = -6.303602849040556e12
10. x = -1.3078357395609307e13, f(x) = -1.3078357395611045e13
11. x = -6.502391861318355e12, f(x) = -6.502391861316581e12
12. x = -3.1192198419074824e12, f(x) = -3.119219841908697e12
13. x = -8.4151908562171045e12, f(x) = -8.415190856218864e12
14. x = -4.0990635647238574e12, f(x) = -4.0990635647257036e12
15. x = -1.782288204126667e12, f(x) = -1.7822882041251582e12
16. x = -7.477324539897932e12, f(x) = -7.477324539896884e12
17. x = -1.8107521355909223e13, f(x) = -1.81075213

147. x = -1.1491376405195054e18, f(x) = -1.1491376405195054e18
148. x = -2.6930909949635057e18, f(x) = -2.6930909949635057e18
149. x = -5.90557137885816e17, f(x) = -5.90557137885816e17
150. x = -3.17660002184143e17, f(x) = -3.17660002184143e17
151. x = -1.9956686235216534e17, f(x) = -1.9956686235216534e17
152. x = -4.5493139194142195e17, f(x) = -4.5493139194142195e17
153. x = -1.5251349479677722e18, f(x) = -1.5251349479677722e18
154. x = -3.939736801674666e18, f(x) = -3.939736801674666e18
155. x = -2.5961850060258903e18, f(x) = -2.5961850060258903e18
156. x = -9.392557400728952e17, f(x) = -9.392557400728952e17
157. x = -2.4733878607488645e18, f(x) = -2.4733878607488645e18
158. x = -1.197322658246067e18, f(x) = -1.197322658246067e18
159. x = -7.90780742367365e18, f(x) = -7.90780742367365e18
160. x = 8.082206736839393e17, f(x) = 8.082206736839393e17
161. x = 1.7168881657173696e18, f(x) = 1.7168881657173696e18
162. x = 2.589612138660444e17, f(x) = 2.589612138660444e17
163. x = 1.466814404

311. x = 488490.5002008055, f(x) = 488492.29769310646
312. x = 228230.37238892887, f(x) = 228230.47376359475
313. x = 457049.1066369943, f(x) = 457050.5169087998
314. x = 268040.326231216, f(x) = 268041.0288571013
315. x = 575244.9755830974, f(x) = 575243.9972134166
316. x = 1.3480481689376961e6, f(x) = 1.3480485389149247e6
317. x = 893468.1565029449, f(x) = 893469.5830783936
318. x = 3.117452982313031e6, f(x) = 3.1174523405247163e6
319. x = 6.603638763609807e6, f(x) = 6.603640763505977e6
320. x = -137378.5083433725, f(x) = -137378.11726846013
321. x = -90988.80526955926, f(x) = -90986.98570739014
322. x = -41273.94389431476, f(x) = -41274.53567707151
323. x = -86608.49775704485, f(x) = -86606.74193403113
324. x = 1.9582138011658492e6, f(x) = 1.958212688315697e6
325. x = 1.222540218473948e6, f(x) = 1.2225384029711015e6
326. x = 557761.9986460645, f(x) = 557762.9528163662
327. x = 355506.46295734413, f(x) = 355507.7463139311
328. x = 215208.48779576246, f(x) = 215208.42030238142
329. x 

473. x = 2.480225386497521e6, f(x) = 2.4802239264708664e6
474. x = 9.24080049464945e6, f(x) = 9.240799566021988e6
475. x = 2.1220965508289173e7, f(x) = 2.1220966219079383e7
476. x = 1.382543785822549e7, f(x) = 1.3825439832516385e7
477. x = -6.49560111535104e6, f(x) = -6.495600791796004e6
478. x = -1.3166962158901673e7, f(x) = -1.316696085434688e7
479. x = -3.868627255539477e7, f(x) = -3.868627264322312e7
480. x = -2.5782549588457823e7, f(x) = -2.5782547643674064e7
481. x = 2.2563451605080374e7, f(x) = 2.25634531763885e7
482. x = 1.1763466656898385e8, f(x) = 1.176346682802809e8
483. x = 5.9832190424334206e7, f(x) = 5.983219194882597e7
484. x = 2.629456880002859e8, f(x) = 2.6294568787539756e8
485. x = 5.269216959695969e8, f(x) = 5.269216964172343e8
486. x = 3.482594515215279e8, f(x) = 3.4825944967078483e8
487. x = -1.0915600132439299e9, f(x) = -1.0915600151834915e9
488. x = 1.0402069453309681e9, f(x) = 1.0402069434810786e9
489. x = -3.297660018127611e9, f(x) = -3.2976600183971972e9
490. 

709. x = -2.4402950899490437e18, f(x) = -2.4402950899490437e18
710. x = -1.5948559623280494e18, f(x) = -1.5948559623280494e18
711. x = -3.193198399512293e18, f(x) = -3.193198399512293e18
712. x = -1.5753400875153646e18, f(x) = -1.5753400875153646e18
713. x = -4.2158040499556536e18, f(x) = -4.2158040499556536e18
714. x = -8.647475552801139e18, f(x) = -8.647475552801139e18
715. x = -3.23864915553646e18, f(x) = -3.23864915553646e18
716. x = -1.7010879874122906e18, f(x) = -1.7010879874122906e18
717. x = -9.260537340458634e17, f(x) = -9.260537340458634e17
718. x = -6.116321042948019e17, f(x) = -6.116321042948019e17
719. x = -3.4839185213430906e17, f(x) = -3.4839185213430906e17
720. x = -2.0815087151235725e17, f(x) = -2.0815087151235725e17
721. x = 4.4891537349523717e18, f(x) = 4.4891537349523717e18
722. x = 2.19976788197085e18, f(x) = 2.19976788197085e18
723. x = -1.4824327713873218e19, f(x) = -1.4824327713873218e19
724. x = -2.981899278073166e19, f(x) = -2.981899278073166e19
725. x = -1.65

855. x = 6.9727057097131e21, f(x) = 6.9727057097131e21
856. x = 1.4056244109094628e22, f(x) = 1.4056244109094628e22
857. x = 6.769462227034111e21, f(x) = 6.769462227034111e21
858. x = 3.4228017107634875e21, f(x) = 3.4228017107634875e21
859. x = -1.0017805055307404e22, f(x) = -1.0017805055307404e22
860. x = -2.0196372881267587e22, f(x) = -2.0196372881267587e22
861. x = -4.948385403787929e22, f(x) = -4.948385403787929e22
862. x = -2.71538434555032e22, f(x) = -2.71538434555032e22
863. x = -1.6286452921170972e22, f(x) = -1.6286452921170972e22
864. x = -4.763396312975515e22, f(x) = -4.763396312975515e22
865. x = -1.1783656279355981e23, f(x) = -1.1783656279355981e23
866. x = -7.661913536014524e22, f(x) = -7.661913536014524e22
867. x = -1.583360818107076e23, f(x) = -1.583360818107076e23
868. x = -1.0545184233445253e23, f(x) = -1.0545184233445253e23
869. x = -6.328688000202057e22, f(x) = -6.328688000202057e22
870. x = -1.6289664597625312e23, f(x) = -1.6289664597625312e23
871. x = -1.1065105289

5.169201968247818e30

Nous sommes moins chanceux! En fait, en $\pi/3$,

In [40]:
df(π/3)

-2.220446049250313e-16

La récurrence de Newton rencontre des problèmes comme nous avons une division par zéro. La fonction continue néanmoins comme, en raison des erreurs numériques, nous évitons la division par zéro. Toutefois, la méthode diverge, comme $|x| \rightarrow \infty$.

Prenons à présent un point où la dérivée est plus importante.

In [41]:
x0 = 4.0
NewtonRoot(f, df, x0, true)

1. x = 4.0, f(x) = 5.513604990615857
2. x = 1.6103517191520549, f(x) = -0.38808385577811944
3. x = 1.9699916004393636, f(x) = 0.12724345855926944
4. x = 1.8984000905205811, f(x) = 0.004767869755435639
5. x = 1.8954991329401036, f(x) = 7.970595869544184e-6
6. x = 1.89549426704768, f(x) = 2.243960572911874e-11


1.89549426704768

La méthode converge à présent très rapidement, même si le point de départ était plus éloigné de la solution.

Notons que $x-2\sin x = 0$ est équivalent à $\frac{1}{\sin x} - \frac{2}{x} = 0$ si nous exigeons que $x \ne k\pi$, $k \in \mathcal{Z}$. La forme de la fonction est cependant différente autour du zéro de la fonction.

In [42]:
g(x) = 1/sin(x) - 2/x
plot(g, 0.1, π-0.1)

<!DOCTYPE html>
 
 
 Plots.jl

La dérivée est

In [44]:
dg(x) = 2/(x*x)-cos(x)/(sin(x)^2)
plot(dg, 0.1, π-0.1)

<!DOCTYPE html>
 
 
 Plots.jl

In [45]:
x0 = 1.1
NewtonRoot(g, dg, x0, true)

1. x = 1.1, f(x) = -0.6961084996546181
2. x = 1.7434765982344966, f(x) = -0.13203651391349291
3. x = 1.9016026742732137, f(x) = 0.005583091094633952
4. x = 1.8955089071540192, f(x) = 1.3349284495589586e-5
5. x = 1.8954942671170651, f(x) = 7.575806648674188e-11


1.8954942671170651

Nous observons maintenant une convergence rapide, principalement en raison du fait que la fonction ne présente pas de parties plates.

In [46]:
f(x) = exp(x/2)-x-1

f (generic function with 1 method)

In [47]:
df(x) = 0.5*exp(x/2)-1

df (generic function with 1 method)

In [48]:
x0 = 1.0
NewtonRoot(f, df, x0, true)

1. x = 1.0, f(x) = -0.3512787292998718
2. x = -1.0, f(x) = 0.6065306597126334
3. x = -0.12946680321283233, f(x) = 0.06678412057450744
4. x = -0.0037771280810190433, f(x) = 0.0018903462554580308
5. x = -3.5554783072134187e-6, f(x) = 1.7777407337327134e-6
6. x = -3.1604497525000708e-12, f(x) = 1.5802914532514478e-12


-3.1604497525000708e-12

Vérifions les résultats obtenus avec la librairie `Roots`.

In [49]:
using Roots

In [50]:
y = fzero(df,0.0, 5.0)

1.3862943611198906

In [51]:
x0 = y
NewtonRoot(f, df, x0, true)

1. x = 1.3862943611198906, f(x) = -0.3862943611198906
2. x = Inf, f(x) = NaN


Inf

In [52]:
df(y)

0.0

## Cycles

Considérons à présent la fonction

In [53]:
h(x) = x^3 - 2*x + 2

h (generic function with 1 method)

In [54]:
plot(h, -2.5, 1.5)

<!DOCTYPE html>
 
 
 Plots.jl

In [55]:
dh(x) = 3x^2-2

dh (generic function with 1 method)

In [56]:
x0 = 0.0
NewtonRoot(h, dh, x0, true)

1. x = 0.0, f(x) = 2.0
2. x = 1.0, f(x) = 1.0
3. x = 0.0, f(x) = 2.0
4. x = 1.0, f(x) = 1.0
5. x = 0.0, f(x) = 2.0
6. x = 1.0, f(x) = 1.0
7. x = 0.0, f(x) = 2.0
8. x = 1.0, f(x) = 1.0
9. x = 0.0, f(x) = 2.0
10. x = 1.0, f(x) = 1.0
11. x = 0.0, f(x) = 2.0
12. x = 1.0, f(x) = 1.0
13. x = 0.0, f(x) = 2.0
14. x = 1.0, f(x) = 1.0
15. x = 0.0, f(x) = 2.0
16. x = 1.0, f(x) = 1.0
17. x = 0.0, f(x) = 2.0
18. x = 1.0, f(x) = 1.0
19. x = 0.0, f(x) = 2.0
20. x = 1.0, f(x) = 1.0
21. x = 0.0, f(x) = 2.0
22. x = 1.0, f(x) = 1.0
23. x = 0.0, f(x) = 2.0
24. x = 1.0, f(x) = 1.0
25. x = 0.0, f(x) = 2.0
26. x = 1.0, f(x) = 1.0
27. x = 0.0, f(x) = 2.0
28. x = 1.0, f(x) = 1.0
29. x = 0.0, f(x) = 2.0
30. x = 1.0, f(x) = 1.0
31. x = 0.0, f(x) = 2.0
32. x = 1.0, f(x) = 1.0
33. x = 0.0, f(x) = 2.0
34. x = 1.0, f(x) = 1.0
35. x = 0.0, f(x) = 2.0
36. x = 1.0, f(x) = 1.0
37. x = 0.0, f(x) = 2.0
38. x = 1.0, f(x) = 1.0
39. x = 0.0, f(x) = 2.0
40. x = 1.0, f(x) = 1.0
41. x = 0.0, f(x) = 2.0
42. x = 1.0, f(x) = 1.0
4

342. x = 1.0, f(x) = 1.0
343. x = 0.0, f(x) = 2.0
344. x = 1.0, f(x) = 1.0
345. x = 0.0, f(x) = 2.0
346. x = 1.0, f(x) = 1.0
347. x = 0.0, f(x) = 2.0
348. x = 1.0, f(x) = 1.0
349. x = 0.0, f(x) = 2.0
350. x = 1.0, f(x) = 1.0
351. x = 0.0, f(x) = 2.0
352. x = 1.0, f(x) = 1.0
353. x = 0.0, f(x) = 2.0
354. x = 1.0, f(x) = 1.0
355. x = 0.0, f(x) = 2.0
356. x = 1.0, f(x) = 1.0
357. x = 0.0, f(x) = 2.0
358. x = 1.0, f(x) = 1.0
359. x = 0.0, f(x) = 2.0
360. x = 1.0, f(x) = 1.0
361. x = 0.0, f(x) = 2.0
362. x = 1.0, f(x) = 1.0
363. x = 0.0, f(x) = 2.0
364. x = 1.0, f(x) = 1.0
365. x = 0.0, f(x) = 2.0
366. x = 1.0, f(x) = 1.0
367. x = 0.0, f(x) = 2.0
368. x = 1.0, f(x) = 1.0
369. x = 0.0, f(x) = 2.0
370. x = 1.0, f(x) = 1.0
371. x = 0.0, f(x) = 2.0
372. x = 1.0, f(x) = 1.0
373. x = 0.0, f(x) = 2.0
374. x = 1.0, f(x) = 1.0
375. x = 0.0, f(x) = 2.0
376. x = 1.0, f(x) = 1.0
377. x = 0.0, f(x) = 2.0
378. x = 1.0, f(x) = 1.0
379. x = 0.0, f(x) = 2.0
380. x = 1.0, f(x) = 1.0
381. x = 0.0, f(x) = 2.0


708. x = 1.0, f(x) = 1.0
709. x = 0.0, f(x) = 2.0
710. x = 1.0, f(x) = 1.0
711. x = 0.0, f(x) = 2.0
712. x = 1.0, f(x) = 1.0
713. x = 0.0, f(x) = 2.0
714. x = 1.0, f(x) = 1.0
715. x = 0.0, f(x) = 2.0
716. x = 1.0, f(x) = 1.0
717. x = 0.0, f(x) = 2.0
718. x = 1.0, f(x) = 1.0
719. x = 0.0, f(x) = 2.0
720. x = 1.0, f(x) = 1.0
721. x = 0.0, f(x) = 2.0
722. x = 1.0, f(x) = 1.0
723. x = 0.0, f(x) = 2.0
724. x = 1.0, f(x) = 1.0
725. x = 0.0, f(x) = 2.0
726. x = 1.0, f(x) = 1.0
727. x = 0.0, f(x) = 2.0
728. x = 1.0, f(x) = 1.0
729. x = 0.0, f(x) = 2.0
730. x = 1.0, f(x) = 1.0
731. x = 0.0, f(x) = 2.0
732. x = 1.0, f(x) = 1.0
733. x = 0.0, f(x) = 2.0
734. x = 1.0, f(x) = 1.0
735. x = 0.0, f(x) = 2.0
736. x = 1.0, f(x) = 1.0
737. x = 0.0, f(x) = 2.0
738. x = 1.0, f(x) = 1.0
739. x = 0.0, f(x) = 2.0
740. x = 1.0, f(x) = 1.0
741. x = 0.0, f(x) = 2.0
742. x = 1.0, f(x) = 1.0
743. x = 0.0, f(x) = 2.0
744. x = 1.0, f(x) = 1.0
745. x = 0.0, f(x) = 2.0
746. x = 1.0, f(x) = 1.0
747. x = 0.0, f(x) = 2.0


1.0

In [57]:
ratio(x) = h(x)/dh(x)

ratio (generic function with 1 method)

In [58]:
ratio(0.0)

-1.0

In [59]:
ratio(1.0)

1.0

La méthode cycle! Cependant, si nous changeons le point de départ, nous pouvons converger.

In [60]:
x0 = 0.5
NewtonRoot(h, dh, x0, true)

1. x = 0.5, f(x) = 1.125
2. x = 1.4, f(x) = 1.9439999999999995
3. x = 0.8989690721649484, f(x) = 0.9285595695281881
4. x = -1.2887793276654596, f(x) = 2.4369578531870273
5. x = -2.105767299013565, f(x) = -3.1259765087015694
6. x = -1.8291999504602496, f(x) = -0.46205275489156605
7. x = -1.771715812062107, f(x) = -0.01794341645386366
8. x = -1.76929656115579, f(x) = -3.1094202426196205e-5
9. x = -1.769292354251341, f(x) = -9.39395228272133e-11


-1.769292354251341

## Application: calcul de la racine carrée d'un nombre non négatif

La racine carrée d'un nombre réel non négatif peut être obtenue à l'aide de la méthode de Newton. Plus précisément, si nous cherchons la racine carrée de $s$, nous pouvons reformuler le problème comme la recherche d'un zéro de la fonction
$$
f(x) = x^2 - s
$$
En effet, si $f(x) = 0$, alors $s = x^2$, ou $x = \pm\sqrt{s}$.

Développons la récurrence de Newton pour ce problème.

Choisir un bon point de départ peut cependant être un problème. (Wikipedia) Avec $s$ exprimé en notation scientifique comme  $a\times 10^{2n}$ avec $1\leq a<100$ et $n$ entier, $\sqrt{s} \approx \sqrt{a} 10^n$. Le point de départ est souvent calculé comme
$$
\sqrt{s} =
\begin{cases}
2\times10^n & \mbox{ if } a < 10\\
6\times10^n & \mbox{ if } a \geq 10
\end{cases}
$$
Les facteurs deux et six sont utilisés car ils permettent d'approximer les moyennes géométriques de la plus petite et la plus grande valeurs pour un nombre donné de chiffres:
$$
\sqrt{\sqrt{1}\sqrt{10}} = 10^{\frac{1}{4}} \approx 2
$$
et
$$
\sqrt{\sqrt{10}\sqrt{100}} = 10^{\frac{3}{4}} \approx 6
$$

In [63]:
sqs(x) = x*x-s

sqs (generic function with 1 method)

In [62]:
dsq(x) = 2*x

dsq (generic function with 1 method)

In [64]:
s = 6.0

6.0

In [65]:
x = NewtonRoot(sqs, dsq, 2.0, true)

1. x = 2.0, f(x) = -2.0
2. x = 2.5, f(x) = 0.25
3. x = 2.45, f(x) = 0.002500000000001279
4. x = 2.4494897959183675, f(x) = 2.603082052132777e-7


2.4494897959183675

Nous pouvons vérifier la solution comme

In [66]:
x^2

6.000000260308205

In [68]:
sqrt(6.0)

2.449489742783178

In [69]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-8)

1. x = 2.0, f(x) = -2.0
2. x = 2.5, f(x) = 0.25
3. x = 2.45, f(x) = 0.002500000000001279
4. x = 2.4494897959183675, f(x) = 2.603082052132777e-7
5. x = 2.4494897427831788, f(x) = 3.552713678800501e-15


2.4494897427831788

In [70]:
x^2

6.0000000000000036

In [71]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-10)
x^2

1. x = 2.0, f(x) = -2.0
2. x = 2.5, f(x) = 0.25
3. x = 2.45, f(x) = 0.002500000000001279
4. x = 2.4494897959183675, f(x) = 2.603082052132777e-7
5. x = 2.4494897427831788, f(x) = 3.552713678800501e-15


6.0000000000000036

In [72]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-12)
x^2

1. x = 2.0, f(x) = -2.0
2. x = 2.5, f(x) = 0.25
3. x = 2.45, f(x) = 0.002500000000001279
4. x = 2.4494897959183675, f(x) = 2.603082052132777e-7
5. x = 2.4494897427831788, f(x) = 3.552713678800501e-15


6.0000000000000036

In [73]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-14)
x^2

1. x = 2.0, f(x) = -2.0
2. x = 2.5, f(x) = 0.25
3. x = 2.45, f(x) = 0.002500000000001279
4. x = 2.4494897959183675, f(x) = 2.603082052132777e-7
5. x = 2.4494897427831788, f(x) = 3.552713678800501e-15


6.0000000000000036

In [74]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-16)
x^2

1. x = 2.0, f(x) = -2.0
2. x = 2.5, f(x) = 0.25
3. x = 2.45, f(x) = 0.002500000000001279
4. x = 2.4494897959183675, f(x) = 2.603082052132777e-7
5. x = 2.4494897427831788, f(x) = 3.552713678800501e-15
6. x = 2.449489742783178, f(x) = -8.881784197001252e-16
7. x = 2.449489742783178, f(x) = -8.881784197001252e-16
8. x = 2.449489742783178, f(x) = -8.881784197001252e-16
9. x = 2.449489742783178, f(x) = -8.881784197001252e-16
10. x = 2.449489742783178, f(x) = -8.881784197001252e-16
11. x = 2.449489742783178, f(x) = -8.881784197001252e-16
12. x = 2.449489742783178, f(x) = -8.881784197001252e-16
13. x = 2.449489742783178, f(x) = -8.881784197001252e-16
14. x = 2.449489742783178, f(x) = -8.881784197001252e-16
15. x = 2.449489742783178, f(x) = -8.881784197001252e-16
16. x = 2.449489742783178, f(x) = -8.881784197001252e-16
17. x = 2.449489742783178, f(x) = -8.881784197001252e-16
18. x = 2.449489742783178, f(x) = -8.881784197001252e-16
19. x = 2.449489742783178, f(x) = -8.881784197001252e-16
20. x 

187. x = 2.449489742783178, f(x) = -8.881784197001252e-16
188. x = 2.449489742783178, f(x) = -8.881784197001252e-16
189. x = 2.449489742783178, f(x) = -8.881784197001252e-16
190. x = 2.449489742783178, f(x) = -8.881784197001252e-16
191. x = 2.449489742783178, f(x) = -8.881784197001252e-16
192. x = 2.449489742783178, f(x) = -8.881784197001252e-16
193. x = 2.449489742783178, f(x) = -8.881784197001252e-16
194. x = 2.449489742783178, f(x) = -8.881784197001252e-16
195. x = 2.449489742783178, f(x) = -8.881784197001252e-16
196. x = 2.449489742783178, f(x) = -8.881784197001252e-16
197. x = 2.449489742783178, f(x) = -8.881784197001252e-16
198. x = 2.449489742783178, f(x) = -8.881784197001252e-16
199. x = 2.449489742783178, f(x) = -8.881784197001252e-16
200. x = 2.449489742783178, f(x) = -8.881784197001252e-16
201. x = 2.449489742783178, f(x) = -8.881784197001252e-16
202. x = 2.449489742783178, f(x) = -8.881784197001252e-16
203. x = 2.449489742783178, f(x) = -8.881784197001252e-16
204. x = 2.449

339. x = 2.449489742783178, f(x) = -8.881784197001252e-16
340. x = 2.449489742783178, f(x) = -8.881784197001252e-16
341. x = 2.449489742783178, f(x) = -8.881784197001252e-16
342. x = 2.449489742783178, f(x) = -8.881784197001252e-16
343. x = 2.449489742783178, f(x) = -8.881784197001252e-16
344. x = 2.449489742783178, f(x) = -8.881784197001252e-16
345. x = 2.449489742783178, f(x) = -8.881784197001252e-16
346. x = 2.449489742783178, f(x) = -8.881784197001252e-16
347. x = 2.449489742783178, f(x) = -8.881784197001252e-16
348. x = 2.449489742783178, f(x) = -8.881784197001252e-16
349. x = 2.449489742783178, f(x) = -8.881784197001252e-16
350. x = 2.449489742783178, f(x) = -8.881784197001252e-16
351. x = 2.449489742783178, f(x) = -8.881784197001252e-16
352. x = 2.449489742783178, f(x) = -8.881784197001252e-16
353. x = 2.449489742783178, f(x) = -8.881784197001252e-16
354. x = 2.449489742783178, f(x) = -8.881784197001252e-16
355. x = 2.449489742783178, f(x) = -8.881784197001252e-16
356. x = 2.449

555. x = 2.449489742783178, f(x) = -8.881784197001252e-16
556. x = 2.449489742783178, f(x) = -8.881784197001252e-16
557. x = 2.449489742783178, f(x) = -8.881784197001252e-16
558. x = 2.449489742783178, f(x) = -8.881784197001252e-16
559. x = 2.449489742783178, f(x) = -8.881784197001252e-16
560. x = 2.449489742783178, f(x) = -8.881784197001252e-16
561. x = 2.449489742783178, f(x) = -8.881784197001252e-16
562. x = 2.449489742783178, f(x) = -8.881784197001252e-16
563. x = 2.449489742783178, f(x) = -8.881784197001252e-16
564. x = 2.449489742783178, f(x) = -8.881784197001252e-16
565. x = 2.449489742783178, f(x) = -8.881784197001252e-16
566. x = 2.449489742783178, f(x) = -8.881784197001252e-16
567. x = 2.449489742783178, f(x) = -8.881784197001252e-16
568. x = 2.449489742783178, f(x) = -8.881784197001252e-16
569. x = 2.449489742783178, f(x) = -8.881784197001252e-16
570. x = 2.449489742783178, f(x) = -8.881784197001252e-16
571. x = 2.449489742783178, f(x) = -8.881784197001252e-16
572. x = 2.449

707. x = 2.449489742783178, f(x) = -8.881784197001252e-16
708. x = 2.449489742783178, f(x) = -8.881784197001252e-16
709. x = 2.449489742783178, f(x) = -8.881784197001252e-16
710. x = 2.449489742783178, f(x) = -8.881784197001252e-16
711. x = 2.449489742783178, f(x) = -8.881784197001252e-16
712. x = 2.449489742783178, f(x) = -8.881784197001252e-16
713. x = 2.449489742783178, f(x) = -8.881784197001252e-16
714. x = 2.449489742783178, f(x) = -8.881784197001252e-16
715. x = 2.449489742783178, f(x) = -8.881784197001252e-16
716. x = 2.449489742783178, f(x) = -8.881784197001252e-16
717. x = 2.449489742783178, f(x) = -8.881784197001252e-16
718. x = 2.449489742783178, f(x) = -8.881784197001252e-16
719. x = 2.449489742783178, f(x) = -8.881784197001252e-16
720. x = 2.449489742783178, f(x) = -8.881784197001252e-16
721. x = 2.449489742783178, f(x) = -8.881784197001252e-16
722. x = 2.449489742783178, f(x) = -8.881784197001252e-16
723. x = 2.449489742783178, f(x) = -8.881784197001252e-16
724. x = 2.449

875. x = 2.449489742783178, f(x) = -8.881784197001252e-16
876. x = 2.449489742783178, f(x) = -8.881784197001252e-16
877. x = 2.449489742783178, f(x) = -8.881784197001252e-16
878. x = 2.449489742783178, f(x) = -8.881784197001252e-16
879. x = 2.449489742783178, f(x) = -8.881784197001252e-16
880. x = 2.449489742783178, f(x) = -8.881784197001252e-16
881. x = 2.449489742783178, f(x) = -8.881784197001252e-16
882. x = 2.449489742783178, f(x) = -8.881784197001252e-16
883. x = 2.449489742783178, f(x) = -8.881784197001252e-16
884. x = 2.449489742783178, f(x) = -8.881784197001252e-16
885. x = 2.449489742783178, f(x) = -8.881784197001252e-16
886. x = 2.449489742783178, f(x) = -8.881784197001252e-16
887. x = 2.449489742783178, f(x) = -8.881784197001252e-16
888. x = 2.449489742783178, f(x) = -8.881784197001252e-16
889. x = 2.449489742783178, f(x) = -8.881784197001252e-16
890. x = 2.449489742783178, f(x) = -8.881784197001252e-16
891. x = 2.449489742783178, f(x) = -8.881784197001252e-16
892. x = 2.449

5.999999999999999

In [75]:
x = NewtonRoot(sqs, dsq, 2.0, true, 1e-15)
x^2

1. x = 2.0, f(x) = -2.0
2. x = 2.5, f(x) = 0.25
3. x = 2.45, f(x) = 0.002500000000001279
4. x = 2.4494897959183675, f(x) = 2.603082052132777e-7
5. x = 2.4494897427831788, f(x) = 3.552713678800501e-15
6. x = 2.449489742783178, f(x) = -8.881784197001252e-16


5.999999999999999

In [76]:
6-x^2

8.881784197001252e-16

In [81]:
6-sqrt(6)^2

8.881784197001252e-16

In [77]:
eps()

2.220446049250313e-16

In [78]:
s = 25
x = NewtonRoot(sqs, dsq, 6.0, true, 1e-15)
x^2

1. x = 6.0, f(x) = 11.0
2. x = 5.083333333333333, f(x) = 0.840277777777775
3. x = 5.0006830601092895, f(x) = 0.006831067664009538
4. x = 5.000000046650738, f(x) = 4.6650738738662767e-7
5. x = 5.0, f(x) = 0.0


25.0

In [79]:
s = 400
x = NewtonRoot(sqs, dsq, 6.0, true, 1e-15)
x^2

1. x = 6.0, f(x) = -364.0
2. x = 36.33333333333333, f(x) = 920.1111111111109
3. x = 23.671253822629964, f(x) = 160.32825753537372
4. x = 20.284693509079986, f(x) = 11.468790757311695
5. x = 20.00199782151197, f(x) = 0.07991685176961028
6. x = 20.000000099772304, f(x) = 3.990892196270579e-6
7. x = 20.0, f(x) = 0.0


400.0